# Info from the web

**This notebook goes with [a blog post at Agile*](http://ageo.co/xlines02).**

We're going to get some info from a web service, and from Wikipedia. We'll make good use of [the `requests` library](http://docs.python-requests.org/en/master/), a really nicely designed Python library for making web requests in Python.

## Curvenam.es

[`curvenam.es`](http://curvenam.es) is a little web app for looking up curve mnemonics from LAS files.

Here's what [the demo request from the site](http://curvenam.es/lookup) looks like:

    http://curvenam.es/lookup?mnemonic=TEST&method=fuzzy&limit=5&maxdist=2
    
We split this into the URL, and the query parameters:

In [ ]:
import requests

In [ ]:
url = 'http://curvenam.es/lookup'

In [ ]:
params = {'mnemonic': 'DT4P',
          'method': 'fuzzy',
          'limit': 1,
          'maxdist': 2
         }

In [ ]:
r = requests.get(url, params)

If we were successful, the server sends back a `200` status code:

In [ ]:
r.status_code

The result of the query is in the `text` attribute of the result:

In [ ]:
r.text

There's a convenient `json()` method to give us the result as JSON:

In [ ]:
r.json()

In [ ]:
try:
    print(r.json()['result'][0]['curve']['description'])
except:
    print("No results")

----

## Geological ages from Wikipedia

Sometimes there isn't a nice API and we have to get what we want from unstructured data. Let's use the task of getting geological ages from Wikipedia as an example.

We'll start with the Jurassic, then generalize.

In [ ]:
url = "http://en.wikipedia.org/wiki/Jurassic"  # Line 1

I used `View Source` in my browser to figure out where the age range is on the page, and what it looks like. The most predictable spot, that will work on every period's page, is in the infobox. It's given as a range, in italic text, with "million years ago" right after it.

Try to find the same string here.

In [ ]:
r = requests.get(url)  # Line 2

Now we have the entire text of the webpage, along with some metadata. The text is stored in `r.text`, and I happen to know roughly where the relevant bit of text is: around the 8500th character, give or take:

In [ ]:
r.text[8400:8600]

In [ ]:
r.text[7400:7600]  # I don't count these lines either.

We can get at that bit of text using a [regular expression](https://docs.python.org/2/library/re.html):

In [ ]:
import re

s = re.search(r'<i>(.+?million years ago)</i>', r.text)
text = s.group(1)
text

And if we're really cunning, we can get the start and end ages:

In [ ]:
start, end = re.search(r'<i>([\.0-9]+)–([\.0-9]+)&#160;million years ago</i>', r.text).groups()  # Line 3
duration = float(start) - float(end)  # Line 4

print("According to Wikipedia, the Jurassic lasted {:.2f} Ma.".format(duration))  # Line 5

An exercise for you, dear reader: Make a function to get the start and end ages of *any* geologic period, taking the name of the period as an argument. I have left some hints.

In [ ]:
def get_age(period):
    url =  "http://en.wikipedia.org/wiki/" + period
    r = requests.get(url)
    start, end =   re.search(r'<i>([\.0-9]+)–([\.0-9]+)&#160;million years ago</i>', r.text).groups()
    return float(start), float(end)

You should be able to call your function like this:

In [ ]:
period = "Jurassic"
get_age(period)

Now we can make a function that makes the sentence we made before, calling the function you just wrote:

In [ ]:
def duration(period):
    t0, t1 = get_age(period)
    duration = t0 - t1
    response = "According to Wikipedia, the {0} lasted {1:.2f} Ma.".format(period, duration)
    return response

In [ ]:
duration('Cretaceous')

<hr />

<div>
<img src="https://avatars1.githubusercontent.com/u/1692321?s=50"><p style="text-align:center">© Agile Geoscience 2016</p>
</div>